In [1]:
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time
from queue import Queue
from ARGO_txt_processor import TXTProcessor
from ARGO_TCP_client import Socket_Client

In [2]:
class MyHandler(FileSystemEventHandler):
    def __init__(self, queue):
        super().__init__()
        self.queue = queue

    def on_created(self, event):
        if not event.is_directory and event.src_path.endswith('.txt'):
            print("New TXT file detected:", event.src_path)
            self.queue.put(event.src_path)  # 将文件路径放入队列中
            
def watch_folder(folder_path, new_path, server_ip, server_port):
    queue = Queue()
    event_handler = MyHandler(queue)
    observer = Observer()
    observer.schedule(event_handler, folder_path, recursive=False)
    observer.start()
    print("Watching folder for new TXT files...")
    
    try:
        while True:
            if not queue.empty():
                file_path = queue.get()
                print(f"Processed file: {file_path}")

                data, DRIFT, profile_data, deviceID, profileID, Argo_SN =  TXTProcessor.extract_data(file_path=file_path)
                new_path = new_path 
                nc_path = TXTProcessor.nc_generate(data,DRIFT,profile_data,new_path,Argo_Type = 'HM-B',deviceID = deviceID,profileID = profileID,lon2 = 105.21,lat2 = 15.11,deploy_time =202303270530,Argo_SN = Argo_SN,ReleaseTime = '2022-12-11 14:03:07 UTC',ReleaseLongitude = '111.8444°E',ReleaseLatitude = '17.4169°N',information = '')
                
                # 在这里调用客户端发送文件的函数，并传递文件路径、服务端IP和端口号
                s = Socket_Client()
                s.socket_client(nc_path, server_ip, server_port)
                
                print('文件发送完毕')
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


In [8]:
folder_path = r'C:\Users\lenovo\Desktop\319485\test'  # 监控文件夹的路径
new_path = r'C:\Users\lenovo\Desktop\319485\nc_test'  # 新文件转换后保存的路径
server_ip = '127.0.0.1'  # 服务端的 IP 地址
server_port = 8001  # 服务端的端口号

# 启动监控文件夹并发送文件到服务端
watch_folder(folder_path, new_path, server_ip, server_port)

Watching folder for new TXT files...
New TXT file detected: C:\Users\lenovo\Desktop\319485\test\39-319485-004.txt
New TXT file detected: C:\Users\lenovo\Desktop\319485\test\39-319485-004.txt
Processed file: C:\Users\lenovo\Desktop\319485\test\39-319485-004.txt
DRIFT为空
Connected to server.
Argo_HM-B_900299_Pro4_105.21E_15.11N_202303270530_Q.nc 数据传输完成...
文件发送完毕
New TXT file detected:New TXT file detected: C:\Users\lenovo\Desktop\319485\test\39-319485-006.txt
 C:\Users\lenovo\Desktop\319485\test\39-319485-006.txt
Processed file: C:\Users\lenovo\Desktop\319485\test\39-319485-006.txt
DRIFT为空
Connected to server.
Argo_HM-B_900299_Pro6_105.21E_15.11N_202303270530_Q.nc 数据传输完成...
文件发送完毕
